In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from datetime import datetime
import random

<h1> get data from database</h1>

In [6]:
import mysql.connector
from mysql.connector import Error
records=[]
try:
    connection = mysql.connector.connect(host='localhost',
                                         database='optimatets',
                                         user='root',
                                         password='')
    sql_select_Query=[]
    sql_select_Query.append( "select * from bookings")
    sql_select_Query.append("select * from driver_vehicle")
    sql_select_Query.append("select * from tracking")
    sql_select_Query.append("select * from vehicles")
    sql_select_Query.append("select * from vehicle_types")
    for req in sql_select_Query:
        cursor = connection.cursor()
        cursor.execute(req)
        records.append( cursor.fetchall())    

except Error as e:
    print("Error reading data from MySQL table", e)
finally:
    if (connection.is_connected()):
        connection.close()
        cursor.close()
        print("MySQL connection is closed")

MySQL connection is closed


In [7]:
bookings=pd.DataFrame(records[0],columns=['id', 'book_type', 'promo_id' , 'type_id' , 'accept_status', 'total_weight', 'company_id', 'date_time', 'user_id', 'vehicle_id', 'driver_id', 'customer_id', 'pickup', 'dropoff', 'duration', 'pickup_addr', 'dest_addr','note', 'travellers', 'status', 'payment', 'created_at', 'updated_at', 'deleted_at'])
bookings.head()

,id,book_type,promo_id,type_id,accept_status,total_weight,company_id,date_time,user_id,vehicle_id,...,duration,pickup_addr,dest_addr,note,travellers,status,payment,created_at,updated_at,deleted_at
0,4,s,NaN,17,53,30.0,11.0,2019-07-03 12:01:13,53,NaN,...,None,"Tunis, Tunisia","Sousse, Tunisia",None,1,0,0,2019-07-01 11:02:24,2019-07-29 12:20:08,2019-07-29 12:20:08
1,8,s,NaN,17,53,20.0,11.0,2019-07-06 07:09:11,53,NaN,...,None,"Cafe Karawen, Route Gremda, Sfax, Tunisia","Avenue De Carthage, Tunisia",None,1,0,0,2019-07-04 14:13:04,2019-07-29 12:20:16,2019-07-29 12:20:16
2,9,s,NaN,6,60,10.0,11.0,2019-07-08 08:21:49,60,7.0,...,None,"tunis, Tunisie","Bizerte Centre, Rue Mohamed Ali, Bizerte, Tunisia",None,1,0,0,2019-07-04 14:23:12,2019-07-29 12:20:25,2019-07-29 12:20:25
3,14,s,NaN,3,51,5.0,10.0,2019-07-11 09:06:34,51,3.0,...,None,"Avenue De Carthage, Tunisie","Sahloul, Sousse, Tunisie",Bien reçu. Je note çà appele moi sur 22222222,1,3,1,2019-07-09 11:08:28,2019-07-29 12:21:40,2019-07-29 12:21:40
4,15,s,NaN,17,60,25.0,11.0,2019-07-14 12:24:05,60,7.0,...,None,"Bizerte, Tunisie","Mahdia, Tunisie",None,1,1,0,2019-07-09 11:24:51,2019-07-29 12:21:47,2019-07-29 12:21:47


In [8]:
df1=bookings
df1.shape

(197, 24)

<h1>add some data to data base </h1>
<h4>to improve the model we added some random rows . so , we picked a random row and we changeed the date  </h4>

In [9]:
def  maxDate(df):
    lista=[]
    for i in range(len(df['date_time'])):
        lista.append(datetime.timestamp(df['date_time'][i]))
    return max(lista)

def addRow(df1,date_time):
    row = df1.sample()
    row['date_time']=date_time
    df1=df1.append(row)
    return df1
            

def fillDatabase(df1):
    now = datetime.timestamp(datetime.now())
    max_date=maxDate(df1)+ 76522
    day = max_date
    cpt=0
    while now> day :
        day=day+86400
        max_date=day+random.randint(39600,86339)
        df1=addRow(df1,datetime.fromtimestamp(max_date))
        cpt=cpt+1
        
        for i in range(random.randint(0,5)):
            max_date=day+random.randint(21600,39600)
            #print(datetime.fromtimestamp(max_date))
            df1=addRow(df1,datetime.fromtimestamp(max_date))
            cpt=cpt+1
            
            for j in range(random.randint(0,3)):
                max_date=day+random.randint(0,21600)
                #print(datetime.fromtimestamp(max_date))
                df1=addRow(df1,datetime.fromtimestamp(max_date))
                cpt=cpt+1
                
    print(cpt ,"rows added !")        
   
                    
                
       
    return df1
   
   
  

In [10]:
df1=fillDatabase(df1)  
df1.shape

657 rows added !


(854, 24)

<h1> adress data normalisation <h1>

In [11]:

region=['Tunis','Manouba','ben arous',
        'ariana',' Bizerte','beja', 'jandouba', 'nabeul' , 'zaghouane', 'silana', 'kef' ,
        'kasserine' , 'kairouane' , 'Sousse' , 'monastir', 'Mahdia' , 'Sfax' , 'sidi' 'bouzid' , 'gafsa' ,
        'touzeur', 'kbeli' , 'gabes' , 'mednine' ,'tataouine']
region=[x.upper() for x in region]
destination_adress= df1['dest_addr'] 
depart_adress = df1['pickup_addr']

In [12]:

def adressNormalisation(col):
    i=0
    lista=[]
    adr=''
    for phrase in col:
        verif=False
        for adrs in region:
            if adrs in phrase.upper():
                verif=True
                adr=adrs
        if verif==True :
                #lista.append([i,phrase,adr])
                lista.append([adr])

        else : 
                #lista.append([i,phrase,random.choice(region)])
                lista.append([random.choice(region)])


        i=i+1


    return pd.DataFrame(lista, columns=['new_'+col.name])



In [13]:

def adressToNan(col):
    i=0
    lista=[]
    adr=''
    for phrase in col:
        verif=False
        for adrs in region:
            if adrs in phrase.upper():
                verif=True
                adr=adrs
        if verif==True :
                #lista.append([i,phrase,adr])
                lista.append([adr])

        else : 
                #lista.append([i,phrase,random.choice(region)])
                lista.append([np.nan])


        i=i+1


    return pd.DataFrame(lista, columns=['new_'+col.name])



In [15]:
df1['pickup_addr']=adressNormalisation(df1['pickup_addr'])
df1['dest_addr']=adressNormalisation(df1['dest_addr'])

<h1> fill the nan vehicle_id </h1>
<h4>this method return a random vihicle_id by adress </h4>

In [16]:
df2=df1
def getVihicleByAdress(df2, adress):
    df2=df2.dropna(subset=["vehicle_id"])
    df2=df2.groupby('pickup_addr')['vehicle_id'].apply(list)
    x=list(set(df2[adress]))
    return random.choice(x)
 
getVihicleByAdress(df2,'SFAX')



21.0

In [17]:
df2.groupby('pickup_addr')['vehicle_id'].apply(list)

pickup_addr
ARIANA     [8.0, 8.0, nan, 8.0, 8.0, 8.0, nan, 8.0, 8.0, ...
BEJA       [23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23.0, 23....
GABES      [nan, nan, nan, nan, 21.0, nan, nan, nan, nan,...
MANOUBA                                                [5.0]
SFAX       [nan, 9.0, nan, nan, nan, 10.0, nan, 10.0, 9.0...
SOUSSE     [nan, nan, nan, 25.0, 21.0, nan, 21.0, 22.0, n...
TUNIS      [nan, 7.0, 3.0, 7.0, nan, nan, nan, 13.0, nan,...
Name: vehicle_id, dtype: object

In [18]:
for i in range(len(df1['vehicle_id'])):
    if np.isnan(df1['vehicle_id'].iloc[i]):
        df1['vehicle_id'].iloc[i]=getVihicleByAdress(df1,df1['pickup_addr'].iloc[i])

D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


<h1> prepare train and test data </h1>

In [20]:
vihicule_id=df1['vehicle_id'].unique()
vihicule_id

array([20., 21.,  7.,  3.,  9., 19., 51., 10., 52., 13., 18., 16., 11.,
       17., 23., 46.,  5., 25., 22.,  8.])

In [21]:
df = pd.DataFrame()
df['Datetime'] = df1['date_time']
for i in range(len(df1)):
    
    df['Year'] = df.Datetime.dt.year
    df['Month'] = df.Datetime.dt.month
    df['Day'] = df.Datetime.dt.day
    df['Hour'] = df.Datetime.dt.hour
    #df['Minute'] = df.Datetime.dt.minute


In [22]:
df1.shape

(854, 24)

In [23]:
y=df1[['dest_addr','pickup_addr']]

In [31]:
x=pd.concat([df[['Year','Day','Month','Hour']],df1['vehicle_id']],sort=False,axis=1)


In [32]:
x.shape

(854, 5)

In [33]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=1)


<h1>train model</h1>

In [34]:

# Compare Algorithms

import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
# load dataset

# prepare configuration for cross validation test harness
seed = 7
# prepare models
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'
for name, model in models:
  
    cv_results = MultiOutputClassifier(model).fit(x_train,y_train)
   
    results.append(cv_results)
    names.append(name)
    msg = "%s   : %s " % (name, cv_results.score(x_test,y_test))
    print(msg)
# boxplot algorithm comparison



D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to ac

LR   : 0.06201550387596899 
LDA   : 0.06201550387596899 
KNN   : 0.3178294573643411 
CART   : 0.43410852713178294 
NB   : 0.20930232558139536 
SVM   : 0.32558139534883723 


In [35]:


def get_vehicle(ids):

    try:
        connection = mysql.connector.connect(host='localhost',
                                         database='optimatets',
                                         user='root',
                                         password='')
        records=[]  
        sql_select_Query=[]
        vihicle=[]
        for i in ids:        
            reqet = "SELECT department_id,company_id,make,model,type_id FROM vehicles WHERE id='" + str(i) + "';"
            sql_select_Query.append(reqet)
       
        for req in sql_select_Query:
            cursor = connection.cursor()
            cursor.execute(req)
            ex=cursor.fetchall()[0]
            msg="this vihicle of the company : "+str(ex[1])+" , departement : "+str(ex[0]) +" with the make : "+ ex[2]+" and a model : "+ex[3]
            records.append(ex)    
            vihicle.append({'message':msg,'type_id':ex[4]})
    except Error as e:
        print("Error reading data from MySQL table", e)
    finally:
        if (connection.is_connected()):
            connection.close()
            cursor.close()
            print("MySQL connection is closed")
    
    return(vihicle)
get_vehicle([4,7,5])       

MySQL connection is closed


[{'message': 'this vihicle of the company : 10 , departement : 3 with the make : Scania and a model : Y6',
  'type_id': 17},
 {'message': 'this vihicle of the company : 11 , departement : 4 with the make : Peugeot and a model : 3008',
  'type_id': 6},
 {'message': 'this vihicle of the company : 11 , departement : 4 with the make : Renault and a model : Piagio',
  'type_id': 17}]

In [ ]:
from flask import Flask, request, make_response
import json
app = Flask(__name__)
@app.route('/', methods=['POST'])
def index():
    
    #ms=str({{"hello":"word"},{"hello":"meher"}})
    d = request.form.to_dict() # data recived from form
    date=d['date'].split("T")
    date1=date[0].split("-")
    hours=date[1].split(':')
    date2=list(map(int, date1)) 
    hours2= list(map(int, hours)) 
   
    
    df=pd.DataFrame(columns=['Datatime'],data=np.array(range(len(vihicule_id))))
   
    for i in range(len(vihicule_id)):  
    
        df['Year'] =date1[0]
        df['Month'] =date1[1]
        df['Day'] = date1[2]
        df['Hour'] =int(hours[0])
        
        
   
    xx=pd.concat([df[['Year','Day','Month','Hour']],pd.Series(vihicule_id).rename("id")],sort=False,axis=1) #input data
    result=pd.concat([pd.DataFrame(results[3].predict(np.array(xx))),xx],sort=False,axis=1)                 #result of prediction
    result=result.rename(columns={0: 'destination',1:'pickup'})
    ms=result[(result.destination == str( d['destination']).upper()) & (result.pickup == str(d['retour']).upper())] # return only the result of our destination and pickup
    if ms.empty:
        vehicles=[]                                              
    else :
        vehicles= get_vehicle(ms['id'].tolist())       # return the data of vehicles predicted  
        
    
   
    
   
    
    print(d)        # data recived from form
    print(result)    # result of the prediction of all vehicles
    print(ms)        # result of the prediction of  vehicle who strat of our pickup and go to destination
    print(vehicles) # the data of vehicle predicted
    
    if vehicles:
        data=make_response(json.dumps(vehicles)) # transform final result to json
    else:
        data=data=make_response(json.dumps([{'message': 'we have not a vehicle in this time . please change the date , pickup or destination', 'type_id': 7}]))
    resp=data
    resp.status_code = 200
    resp.headers['Access-Control-Allow-Origin'] = '*' # configiration 
    return resp


    
    
    
if __name__ == "__main__":
    app.run()
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/May/2020 21:45:25] "POST / HTTP/1.1" 200 -


{'date': '2020-11-12T09:05', 'retour': 'gabes', 'destination': 'mahdia'}
   destination  pickup  Year Day Month  Hour    id
0         SFAX   TUNIS  2020  12    11     9  20.0
1         SFAX    SFAX  2020  12    11     9  21.0
2         SFAX   TUNIS  2020  12    11     9   7.0
3         SFAX   TUNIS  2020  12    11     9   3.0
4        TUNIS    SFAX  2020  12    11     9   9.0
5        TUNIS    SFAX  2020  12    11     9  19.0
6         SFAX   TUNIS  2020  12    11     9  51.0
7        TUNIS    SFAX  2020  12    11     9  10.0
8        GABES   TUNIS  2020  12    11     9  52.0
9         SFAX   TUNIS  2020  12    11     9  13.0
10        SFAX   TUNIS  2020  12    11     9  18.0
11        SFAX   TUNIS  2020  12    11     9  16.0
12        SFAX   TUNIS  2020  12    11     9  11.0
13        SFAX   TUNIS  2020  12    11     9  17.0
14   KASSERINE    BEJA  2020  12    11     9  23.0
15        SFAX   TUNIS  2020  12    11     9  46.0
16        SFAX   TUNIS  2020  12    11     9   5.0
17       